## Restaurant Review Analysis

In [4]:
# Data processing
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

data = pd.read_excel('Sentiment Analysis and Segmentation.xlsx')

# There are some symbols in the reviews we want to get rid of 
def clean(text):
    return text.replace('Œæ', '')
data['review'] = data['review'].apply(clean)

### Traditional Approach: Sentiment Analysis + Topic Modeling

In [116]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
from gensim import corpora
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
# Sentiment analysis using vader. Vader is fast to run and suited for informal texts such as reviews
def get_compound_score(text):
  sid = SentimentIntensityAnalyzer()
  return sid.polarity_scores(text)['compound']

# We want to focus only on the negative reviews so set compund score to less than 0
data['sentiment'] = data['review'].apply(get_compound_score)
negative_reviews = data[data['sentiment']<0]

In [117]:
# Text processing 
wnl = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


def preprocess_text(text):
    # tokenize
    tokens = word_tokenize(text.lower())
    # lemmatize and exclude numbers and stop words
    lemmatized_tokens = [wnl.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    # only include verbs, nouns, adjectives, and adverbs for more accurate analysis
    pos_tags = nltk.pos_tag(lemmatized_tokens) 
    filtered_tokens = [
        word for word, tag in pos_tags
        if tag.startswith('J') or tag.startswith('V') or tag.startswith('N') or tag.startswith('R')
    ]
    return filtered_tokens

negative_reviews['processed_review'] = negative_reviews['review'].apply(preprocess_text)

In [118]:
# Bigrams and trigrams so that we include phrases
data_words = negative_reviews['processed_review']
# Bigram and trigrams phrases
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)
# Change phrases to phraser for better processing
bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

# Make bigrams and trigrams for the reviews
def make_bigrams(texts):
    return([bigram[doc] for doc in texts])
def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])
data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

negative_reviews['trigrams'] = data_bigrams_trigrams

In [121]:
# Dictionary 
dictionary = corpora.Dictionary(negative_reviews['trigrams'])

# Filter out words that occur in less than 5 reviews or more than 50% of the reviews
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Convert the dictionary to a bag of words corpus for each document
corpus = [dictionary.doc2bow(review) for review in negative_reviews['trigrams']]

In [129]:
# Use coherence analysis to find out how many topics are there 
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

# Coherence score for each model
def compute_coherence(model, corpus, dictionary):
    coherence_model = CoherenceModel(model=model, texts=negative_reviews['trigrams'], dictionary=dictionary, coherence='c_v')
    return coherence_model.get_coherence()

# Test from 2 to 20 topics 
coherences = []
for num_topics in range(2, 21): 
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
    coherences.append((num_topics, compute_coherence(lda_model, corpus, dictionary)))

topics = pd.DataFrame(coherences,columns = ['topic','score'])

In [136]:
# We have 4 topics in the negative reviews
topics[topics['score'] == topics['score'].max()]

,topic,score
2,4,0.410031


In [138]:
# Train the LDA model
lda_model = gensim.models.LdaModel(corpus,num_topics=4,id2word=dictionary,passes=15)

# Print the topics and the top words associated with them
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.018*"go" + 0.017*"mcdonald" + 0.016*"food" + 0.015*"location" + 0.015*"time" + 0.012*"get" + 0.012*"mcdonalds" + 0.011*"order" + 0.010*"always" + 0.009*"place"')
(1, '0.021*"food" + 0.020*"mcdonald" + 0.017*"mcdonalds" + 0.014*"time" + 0.013*"service" + 0.012*"place" + 0.010*"go" + 0.010*"always" + 0.010*"people" + 0.009*"drive"')
(2, '0.026*"order" + 0.021*"time" + 0.018*"drive" + 0.016*"get" + 0.016*"go" + 0.013*"mcdonalds" + 0.012*"thru" + 0.011*"never" + 0.011*"ever" + 0.011*"place"')
(3, '0.034*"order" + 0.020*"get" + 0.017*"food" + 0.014*"time" + 0.011*"minute" + 0.010*"window" + 0.010*"drive" + 0.010*"customer" + 0.010*"service" + 0.009*"ordered"')


In [139]:
# Get the topic distribution for each review
negative_reviews['topics'] = [lda_model[corpus[i]] for i in range(len(corpus))]

# Assign the dominant topic to each review
def dominant_topic(topics):
    return max(topics, key=lambda x: x[1])[0]

negative_reviews['dominant_topic'] = negative_reviews['topics'].apply(dominant_topic)

In [144]:
# Check the overal distribution of the topics
negative_reviews['dominant_topic'].value_counts()

,count
dominant_topic,
3,273
1,208
2,153
0,153


In [152]:
# Visualization of the topic categories 
#pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.064471 -0.052161       1        1  41.523372
1     -0.054384  0.072065       2        1  21.881279
2      0.046037 -0.078282       3        1  18.759085
0      0.072818  0.058378       4        1  17.836264, topic_info=         Term        Freq       Total Category  logprob  loglift
11      order  595.000000  595.000000  Default  30.0000  30.0000
89   mcdonald  324.000000  324.000000  Default  29.0000  29.0000
107        go  315.000000  315.000000  Default  28.0000  28.0000
82     always  157.000000  157.000000  Default  27.0000  27.0000
113  location  195.000000  195.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
14      staff   25.099840  107.785897   Topic4  -5.2823   0.2667
111      know   24.655333  106.635402   Topic4  -5.3002   0.2595
305    really   23.978459  105.231182   Topic4  -5.3280   0.2449
475       got   25.221348  168.234582   Topic4  -5.2775  -0.1737
229     right   23.396326  135.070821   Topic4  -5.3526  -0.0293

[320 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
50        1  0.881336      acted
820       1  0.173398  afternoon
820       4  0.693592  afternoon
516       2  0.751386      alone
516       3  0.150277      alone
...     ...       ...        ...
338       3  0.130766      wrong
338       4  0.101707      wrong
1005      1  0.130221       yeah
1005      3  0.781324       yeah
1005      4  0.130221       yeah

[667 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1])

In [145]:
negative_reviews.to_csv('reviews.csv',index=False)

<p style="font-size: 20px; color: black;">
From the 4 segments the model identified, they have the following issues:
<ol style="font-size: 20px; color: black;">
<li>Bad service and high price: workers there are rude to customers and some items have much higher prices than other McDonald's</li>
<li>Wait time is long: for both drive-thru and the counter, customers have to wait a long time to get their orders </li>
<li>Mess up orders: the workers often give wrong and missing orders and sometimes the food is overcooked</li>
<li>Overall dirty and hostile environment: the restaurant is in a dangerous neighborhood with a homeless population. It is often times dirty and the managers often yell at customers and workers.</li>
</ol>
</p>

### LLM Appraoch: Prompting using LLama 3

In [ ]:
# Combine all reviews
reviews = "\n".join(data['review'])

In [7]:
# Use llama3 for privacy reasons
import ollama
response = ollama.chat(
    model="llama3",
    messages=[
        {"role": "user",
        "content": "Read the following reviews of a resturant, and tell me what the main issues of the resturant customers are complaining about"+reviews,
        },
    ],
)
print(response["message"]["content"])

What a treasure trove of frustration! Here are the main issues that stood out to me:

1. **Unprofessional staff**: Multiple reviewers mentioned seeing employees goofing off, bickering with each other, and being generally unprofessional.
2. **Order errors**: A significant number of reviewers experienced mistakes with their orders, such as incorrect items or missing parts.
3. **Rude management**: Some reviewers were dealt with poorly by managers who argued with them about order issues or refused to help with problems.
4. **Dirty environment**: One reviewer mentioned that the restaurant was dirty, with food barely warm and slow service.
5. **Safety concerns**: A few reviewers expressed concerns about safety, citing instances of loitering homeless individuals, gangs, and aggressive panhandling in the area.

Overall, it seems like this McDonald's location has some significant issues with staff behavior, order fulfillment, and customer satisfaction.


<p style="font-size: 20px; color: black;">
The LLM has identified similar issues to our traditional approach. Below is a comparison for both models:</p>

<table style="font-size: 16px; color: black;" border="1" cellpadding="5" cellspacing="5">
    <tr>
        <th></th> 
        <th>Pro</th>
        <th>Con</th>
    </tr>
    <tr>
        <th>Traditional Approach</th>
        <td>Have more control over the data cleaning and modeling</td>
        <td>More technically challenging</td>
    </tr>
    <tr>
        <th>LLM</th>
        <td>Very easy to implement, user-friendly</td>
        <td>Might be computationally expensive on a large scale</td>
    </tr>
</table>